In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from keras.utils.np_utils import to_categorical
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout, Flatten, Lambda, Activation
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam

train_df = pd.read_json('../input/train.json')
test_df = pd.read_json('../input/test.json')

Using TensorFlow backend.


In [2]:
def std_img(x):
    for i in range(3):
        x[:, :, i] -= np.mean(x[:, :, i].flatten())
        x[:, :, i] /= np.std(x[:, :, i].flatten()) + 1e-7
    return x

def get_image(df):
    '''Create 3-channel 'images'. Return rescale-normalised images.'''
    images = []
    for i, row in df.iterrows():
        # Formulate the bands as 75x75 arrays
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        band_3 = (band_1 + band_2)/2

        # Rescale
        img = np.dstack([band_1,band_2,band_3])
        img = std_img(img)

        images.append(img)
    return np.array(images)


train_x = get_image(train_df)
test_x = get_image(test_df)

print(train_x.shape,test_x.shape)

(1604, 75, 75, 3) (8424, 75, 75, 3)


In [7]:
y = train_df.is_iceberg.values
print(y[:5])
from keras.callbacks import ModelCheckpoint,LearningRateScheduler
from keras.layers import Input
from keras.models import Model
from sklearn.model_selection import KFold
from keras.preprocessing.image import ImageDataGenerator
import pickle
from sklearn.metrics import log_loss

def lr_f(epoch):
    if epoch<10:
        return 0.001
    elif epoch<30:
        return 0.0005
    elif epoch<50:
        return 0.0001
    else:
        return 0.00005

[0 0 1 0 0]


In [8]:
from keras.callbacks import ModelCheckpoint
from keras.layers import Input,AveragePooling2D,GlobalMaxPooling2D,GlobalAveragePooling2D,Flatten
from keras import layers
from keras.models import Model

def conv2d_bn(x,
              filters,
              num_row,
              num_col,
              padding='same',
              strides=(1, 1),
              name=None):

    if name is not None:
        bn_name = name + '_bn'
        conv_name = name + '_conv'
    else:
        bn_name = None
        conv_name = None
    bn_axis = 3
    x = Conv2D(
        filters, (num_row, num_col),
        strides=strides,
        padding=padding,
        use_bias=False,
        name=conv_name)(x)
    x = BatchNormalization(axis=bn_axis, scale=False, name=bn_name)(x)
    x = Activation('relu', name=name)(x)
    return x

def create_incept_model():
    img_input = Input(shape=(75,75,3))
    channel_axis = 3
    
    # bn
    x = conv2d_bn(img_input, 32, 3, 3, strides=(2, 2), padding='valid')
    x = conv2d_bn(x, 32, 3, 3, padding='valid')
    x = conv2d_bn(x, 64, 3, 3)
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)

    x = conv2d_bn(x, 64, 1, 1, padding='valid')
    x = conv2d_bn(x, 64, 3, 3, padding='valid')
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)

    # mixed 0, 1, 2: 35 x 35 x 256
    branch1x1 = conv2d_bn(x, 64, 1, 1)

    branch5x5 = conv2d_bn(x, 48, 1, 1)
    branch5x5 = conv2d_bn(branch5x5, 64, 5, 5)

    branch3x3dbl = conv2d_bn(x, 64, 1, 1)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 64, 3, 3)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 64, 3, 3)

    branch_pool = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(x)
    branch_pool = conv2d_bn(branch_pool, 32, 1, 1)
    x = layers.concatenate(
        [branch1x1, branch5x5, branch3x3dbl, branch_pool],
        axis=channel_axis,
        name='mixed0')
    
    # mixed 1: 35 x 35 x 256
    branch1x1 = conv2d_bn(x, 96, 1, 1)

    branch5x5 = conv2d_bn(x, 48, 1, 1)
    branch5x5 = conv2d_bn(branch5x5, 96, 5, 5)

    branch3x3dbl = conv2d_bn(x, 96, 1, 1)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)

    branch_pool = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(x)
    branch_pool = conv2d_bn(branch_pool, 96, 1, 1)
    x = layers.concatenate(
        [branch1x1, branch5x5, branch3x3dbl, branch_pool],
        axis=channel_axis,
        name='mixed1')
    
    # mixed 2: 35 x 35 x 256
    branch1x1 = conv2d_bn(x, 96, 1, 1)

    branch5x5 = conv2d_bn(x, 48, 1, 1)
    branch5x5 = conv2d_bn(branch5x5, 96, 5, 5)

    branch3x3dbl = conv2d_bn(x, 96, 1, 1)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)

    branch_pool = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(x)
    branch_pool = conv2d_bn(branch_pool, 96, 1, 1)
    x = layers.concatenate(
        [branch1x1, branch5x5, branch3x3dbl, branch_pool],
        axis=channel_axis,
        name='mixed2')

    x = Conv2D(96, 3, strides=2, padding='same',activation='relu')(x)
    x = Flatten()(x)
    x = Dense(256,activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(1,activation='sigmoid')(x)
    
    return Model(img_input, x, name='inception_v3')
print('model model')

model model


In [9]:

def kfold_train(fold_cnt=3,rnd=99):
    train_pred, test_pred = np.zeros((1604,1)),np.zeros((8424,1))
    kf = KFold(n_splits=fold_cnt, shuffle=True, random_state=2*rnd)
    for train_index, test_index in kf.split(train_x):
        curr_x,curr_y = train_x[train_index],y[train_index]
        val_x,val_y = train_x[test_index],y[test_index]
        datagen = ImageDataGenerator(
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            zoom_range=0.05,
            horizontal_flip=True,
            vertical_flip=True
        )
        
        
        bat_size = 16
        steps_train = len(curr_y)//bat_size
        
        
        model = create_incept_model()
        model.compile(loss='binary_crossentropy', optimizer=Adam(0.0005), metrics=['accuracy'])
        model_p = 'best_m.h5'
        model_chk = ModelCheckpoint(filepath=model_p, monitor='val_loss', save_best_only=True, verbose=1)
        lr_s = LearningRateScheduler(lr_f)
        model.fit_generator(datagen.flow(curr_x, curr_y, batch_size=bat_size),
                  validation_data=(val_x,val_y),
                  steps_per_epoch = steps_train,
                  epochs=70, 
                  verbose=2,
                  callbacks=[model_chk,lr_s]
                 )
        
        
        model = load_model(model_p)
        train_pred[test_index] = model.predict(val_x)
        test_pred = test_pred + model.predict(test_x)/fold_cnt
        print('============================')
    return train_pred,test_pred

train_pred,test_pred = kfold_train(fold_cnt=4)


with open('../features/incept_aug2_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)

# train feat loss
print(log_loss(y,train_pred))
    
submission = pd.DataFrame()
submission['id']=test_df['id']
submission['is_iceberg']=test_pred
print(submission.head())
submission.to_csv('../results/incept_aug2_sub.csv', index=False)

Epoch 1/70
Epoch 00001: val_loss improved from inf to 0.96025, saving model to best_m.h5
 - 7s - loss: 0.5773 - acc: 0.7258 - val_loss: 0.9603 - val_acc: 0.5461
Epoch 2/70
Epoch 00002: val_loss did not improve
 - 4s - loss: 0.4532 - acc: 0.8125 - val_loss: 1.2365 - val_acc: 0.5536
Epoch 3/70
Epoch 00003: val_loss improved from 0.96025 to 0.63080, saving model to best_m.h5
 - 4s - loss: 0.3974 - acc: 0.8222 - val_loss: 0.6308 - val_acc: 0.7057
Epoch 4/70
Epoch 00004: val_loss did not improve
 - 3s - loss: 0.4194 - acc: 0.8122 - val_loss: 0.8142 - val_acc: 0.7731
Epoch 5/70
Epoch 00005: val_loss improved from 0.63080 to 0.55776, saving model to best_m.h5
 - 4s - loss: 0.3553 - acc: 0.8381 - val_loss: 0.5578 - val_acc: 0.7855
Epoch 6/70
Epoch 00006: val_loss did not improve
 - 4s - loss: 0.3542 - acc: 0.8464 - val_loss: 0.7868 - val_acc: 0.6658
Epoch 7/70
Epoch 00007: val_loss did not improve
 - 4s - loss: 0.3305 - acc: 0.8631 - val_loss: 0.5613 - val_acc: 0.7805
Epoch 8/70
Epoch 00008: v

Epoch 64/70
Epoch 00064: val_loss did not improve
 - 4s - loss: 0.1683 - acc: 0.9250 - val_loss: 0.2658 - val_acc: 0.9002
Epoch 65/70
Epoch 00065: val_loss did not improve
 - 4s - loss: 0.1614 - acc: 0.9339 - val_loss: 0.3051 - val_acc: 0.9002
Epoch 66/70
Epoch 00066: val_loss did not improve
 - 4s - loss: 0.1618 - acc: 0.9308 - val_loss: 0.2674 - val_acc: 0.9027
Epoch 67/70
Epoch 00067: val_loss did not improve
 - 4s - loss: 0.1728 - acc: 0.9258 - val_loss: 0.2832 - val_acc: 0.8953
Epoch 68/70
Epoch 00068: val_loss did not improve
 - 4s - loss: 0.1564 - acc: 0.9306 - val_loss: 0.2891 - val_acc: 0.8978
Epoch 69/70
Epoch 00069: val_loss did not improve
 - 4s - loss: 0.1696 - acc: 0.9283 - val_loss: 0.2832 - val_acc: 0.8928
Epoch 70/70
Epoch 00070: val_loss did not improve
 - 4s - loss: 0.1802 - acc: 0.9267 - val_loss: 0.2702 - val_acc: 0.9002
Epoch 1/70
Epoch 00001: val_loss improved from inf to 0.66196, saving model to best_m.h5
 - 11s - loss: 0.6128 - acc: 0.6895 - val_loss: 0.6620 - 

Epoch 59/70
Epoch 00059: val_loss did not improve
 - 4s - loss: 0.1544 - acc: 0.9367 - val_loss: 0.1842 - val_acc: 0.9152
Epoch 60/70
Epoch 00060: val_loss did not improve
 - 4s - loss: 0.1542 - acc: 0.9425 - val_loss: 0.1920 - val_acc: 0.9102
Epoch 61/70
Epoch 00061: val_loss did not improve
 - 4s - loss: 0.1552 - acc: 0.9375 - val_loss: 0.1974 - val_acc: 0.9102
Epoch 62/70
Epoch 00062: val_loss did not improve
 - 4s - loss: 0.1803 - acc: 0.9325 - val_loss: 0.1907 - val_acc: 0.9152
Epoch 63/70
Epoch 00063: val_loss did not improve
 - 4s - loss: 0.1423 - acc: 0.9417 - val_loss: 0.2165 - val_acc: 0.9077
Epoch 64/70
Epoch 00064: val_loss did not improve
 - 4s - loss: 0.1459 - acc: 0.9383 - val_loss: 0.1870 - val_acc: 0.9177
Epoch 65/70
Epoch 00065: val_loss did not improve
 - 4s - loss: 0.1634 - acc: 0.9325 - val_loss: 0.1901 - val_acc: 0.9202
Epoch 66/70
Epoch 00066: val_loss did not improve
 - 3s - loss: 0.1666 - acc: 0.9333 - val_loss: 0.2014 - val_acc: 0.9152
Epoch 67/70
Epoch 00067:

Epoch 00052: val_loss did not improve
 - 3s - loss: 0.1807 - acc: 0.9175 - val_loss: 0.2161 - val_acc: 0.9227
Epoch 53/70
Epoch 00053: val_loss did not improve
 - 3s - loss: 0.1946 - acc: 0.9166 - val_loss: 0.2378 - val_acc: 0.8928
Epoch 54/70
Epoch 00054: val_loss did not improve
 - 3s - loss: 0.1809 - acc: 0.9292 - val_loss: 0.2228 - val_acc: 0.9052
Epoch 55/70
Epoch 00055: val_loss improved from 0.21358 to 0.21280, saving model to best_m.h5
 - 4s - loss: 0.1639 - acc: 0.9350 - val_loss: 0.2128 - val_acc: 0.9177
Epoch 56/70
Epoch 00056: val_loss did not improve
 - 4s - loss: 0.1802 - acc: 0.9308 - val_loss: 0.2178 - val_acc: 0.9177
Epoch 57/70
Epoch 00057: val_loss improved from 0.21280 to 0.21032, saving model to best_m.h5
 - 4s - loss: 0.1720 - acc: 0.9292 - val_loss: 0.2103 - val_acc: 0.9152
Epoch 58/70
Epoch 00058: val_loss did not improve
 - 4s - loss: 0.1454 - acc: 0.9433 - val_loss: 0.2115 - val_acc: 0.9252
Epoch 59/70
Epoch 00059: val_loss did not improve
 - 4s - loss: 0.1708

Epoch 44/70
Epoch 00044: val_loss did not improve
 - 4s - loss: 0.1886 - acc: 0.9225 - val_loss: 0.2220 - val_acc: 0.9127
Epoch 45/70
Epoch 00045: val_loss did not improve
 - 4s - loss: 0.1650 - acc: 0.9283 - val_loss: 0.2275 - val_acc: 0.8978
Epoch 46/70
Epoch 00046: val_loss did not improve
 - 4s - loss: 0.1690 - acc: 0.9300 - val_loss: 0.2231 - val_acc: 0.9152
Epoch 47/70
Epoch 00047: val_loss did not improve
 - 4s - loss: 0.1909 - acc: 0.9250 - val_loss: 0.2792 - val_acc: 0.8653
Epoch 48/70
Epoch 00048: val_loss did not improve
 - 4s - loss: 0.1832 - acc: 0.9314 - val_loss: 0.2234 - val_acc: 0.9177
Epoch 49/70
Epoch 00049: val_loss did not improve
 - 4s - loss: 0.2099 - acc: 0.9079 - val_loss: 0.2236 - val_acc: 0.9127
Epoch 50/70
Epoch 00050: val_loss did not improve
 - 4s - loss: 0.1830 - acc: 0.9250 - val_loss: 0.2334 - val_acc: 0.9052
Epoch 51/70
Epoch 00051: val_loss did not improve
 - 4s - loss: 0.1910 - acc: 0.9137 - val_loss: 0.2348 - val_acc: 0.9052
Epoch 52/70
Epoch 00052:

In [10]:
from keras.layers import Input,AveragePooling2D,GlobalMaxPooling2D,GlobalAveragePooling2D,Flatten


def identity_block(input_tensor, kernel_size, filters, stage, block):
    """The identity block is the block that has no conv layer at shortcut.
    # Arguments
        input_tensor: input tensor
        kernel_size: default 3, the kernel size of middle conv layer at main path
        filters: list of integers, the filters of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
    # Returns
        Output tensor for the block.
    """
    filters1, filters2, filters3 = filters
    bn_axis = 3
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = Conv2D(filters1, (1, 1), name=conv_name_base + '2a')(input_tensor)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, kernel_size,
               padding='same', name=conv_name_base + '2b')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters3, (1, 1), name=conv_name_base + '2c')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)

    x = layers.add([x, input_tensor])
    x = Activation('relu')(x)
    return x


def conv_block(input_tensor, kernel_size, filters, stage, block, strides=(2, 2)):
    """A block that has a conv layer at shortcut.
    # Arguments
        input_tensor: input tensor
        kernel_size: default 3, the kernel size of middle conv layer at main path
        filters: list of integers, the filters of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
    # Returns
        Output tensor for the block.
    Note that from stage 3, the first conv layer at main path is with strides=(2,2)
    And the shortcut should have strides=(2,2) as well
    """
    filters1, filters2, filters3 = filters

    bn_axis = 3
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = Conv2D(filters1, (1, 1), strides=strides,
               name=conv_name_base + '2a')(input_tensor)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, kernel_size, padding='same',
               name=conv_name_base + '2b')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters3, (1, 1), name=conv_name_base + '2c')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)

    shortcut = Conv2D(filters3, (1, 1), strides=strides,
                      name=conv_name_base + '1')(input_tensor)
    shortcut = BatchNormalization(axis=bn_axis, name=bn_name_base + '1')(shortcut)

    x = layers.add([x, shortcut])
    x = Activation('relu')(x)
    return x

def create_resnet_model():
    img_input = Input(shape=(75,75,3))
    channel_axis = 3
    bn_axis = 3
    x = Conv2D(64, (7, 7), strides=(2, 2), padding='same', name='conv1')(img_input)
    x = BatchNormalization(axis=bn_axis, name='bn_conv1')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)

    x = conv_block(x, 3, [64, 64, 128], stage=2, block='a', strides=(1, 1))
    x = identity_block(x, 3, [64, 64, 128], stage=2, block='b')
    x = identity_block(x, 3, [64, 64, 128], stage=2, block='c')

    x = conv_block(x, 3, [128, 128, 256], stage=3, block='a')
    x = identity_block(x, 3, [128, 128, 256], stage=3, block='b')
    x = identity_block(x, 3, [128, 128, 256], stage=3, block='c')
    x = identity_block(x, 3, [128, 128, 256], stage=3, block='d')
    
    x = Conv2D(128, 3, strides=2, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Flatten()(x)
    x = Dense(256,activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(1,activation='sigmoid')(x)
    
    return Model(img_input, x, name='inception_v3')
print('model model')


model model


In [11]:

def kfold_train(fold_cnt=3,rnd=999):
    train_pred, test_pred = np.zeros((1604,1)),np.zeros((8424,1))
    kf = KFold(n_splits=fold_cnt, shuffle=True, random_state=2*rnd)
    for train_index, test_index in kf.split(train_x):
        curr_x,curr_y = train_x[train_index],y[train_index]
        val_x,val_y = train_x[test_index],y[test_index]
        datagen = ImageDataGenerator(
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            zoom_range=0.05,
            horizontal_flip=True,
            vertical_flip=True
        )
        
        
        bat_size = 16
        steps_train = len(curr_y)//bat_size
        
        
        model = create_resnet_model()
        model.compile(loss='binary_crossentropy', optimizer=Adam(0.0005), metrics=['accuracy'])
        model_p = 'best_m.h5'
        model_chk = ModelCheckpoint(filepath=model_p, monitor='val_loss', save_best_only=True, verbose=1)
        lr_s = LearningRateScheduler(lr_f)
        model.fit_generator(datagen.flow(curr_x, curr_y, batch_size=bat_size),
                  validation_data=(val_x,val_y),
                  steps_per_epoch = steps_train,
                  epochs=70, 
                  verbose=2,
                  callbacks=[model_chk,lr_s]
                 )
        
        
        model = load_model(model_p)
        train_pred[test_index] = model.predict(val_x)
        test_pred = test_pred + model.predict(test_x)/fold_cnt
        print('============================')
    return train_pred,test_pred

train_pred,test_pred = kfold_train(fold_cnt=4)

with open('../features/resnet_aug2_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)

# train feat loss
print(log_loss(y,train_pred))
    
submission = pd.DataFrame()
submission['id']=test_df['id']
submission['is_iceberg']=test_pred
print(submission.head())
submission.to_csv('../results/resnet_aug2_sub.csv', index=False)

Epoch 1/70
Epoch 00001: val_loss improved from inf to 0.69245, saving model to best_m.h5
 - 25s - loss: 1.1433 - acc: 0.5276 - val_loss: 0.6924 - val_acc: 0.5112
Epoch 2/70
Epoch 00002: val_loss did not improve
 - 5s - loss: 0.5866 - acc: 0.6881 - val_loss: 0.7047 - val_acc: 0.4788
Epoch 3/70
Epoch 00003: val_loss improved from 0.69245 to 0.67161, saving model to best_m.h5
 - 5s - loss: 0.5552 - acc: 0.7397 - val_loss: 0.6716 - val_acc: 0.6833
Epoch 4/70
Epoch 00004: val_loss improved from 0.67161 to 0.57105, saving model to best_m.h5
 - 5s - loss: 0.5760 - acc: 0.6931 - val_loss: 0.5711 - val_acc: 0.6858
Epoch 5/70
Epoch 00005: val_loss improved from 0.57105 to 0.49979, saving model to best_m.h5
 - 5s - loss: 0.5156 - acc: 0.7631 - val_loss: 0.4998 - val_acc: 0.8180
Epoch 6/70
Epoch 00006: val_loss did not improve
 - 5s - loss: 0.5013 - acc: 0.7625 - val_loss: 2.5831 - val_acc: 0.6434
Epoch 7/70
Epoch 00007: val_loss improved from 0.49979 to 0.44544, saving model to best_m.h5
 - 5s - 

Epoch 61/70
Epoch 00061: val_loss did not improve
 - 5s - loss: 0.2643 - acc: 0.8950 - val_loss: 0.2283 - val_acc: 0.9302
Epoch 62/70
Epoch 00062: val_loss improved from 0.20233 to 0.19991, saving model to best_m.h5
 - 5s - loss: 0.2634 - acc: 0.9000 - val_loss: 0.1999 - val_acc: 0.9227
Epoch 63/70
Epoch 00063: val_loss did not improve
 - 5s - loss: 0.2824 - acc: 0.8868 - val_loss: 0.2244 - val_acc: 0.9277
Epoch 64/70
Epoch 00064: val_loss did not improve
 - 5s - loss: 0.2629 - acc: 0.8950 - val_loss: 0.2092 - val_acc: 0.9152
Epoch 65/70
Epoch 00065: val_loss did not improve
 - 5s - loss: 0.2923 - acc: 0.8823 - val_loss: 0.2422 - val_acc: 0.9227
Epoch 66/70
Epoch 00066: val_loss did not improve
 - 5s - loss: 0.2827 - acc: 0.9010 - val_loss: 0.2838 - val_acc: 0.9202
Epoch 67/70
Epoch 00067: val_loss did not improve
 - 5s - loss: 0.2588 - acc: 0.8892 - val_loss: 0.2272 - val_acc: 0.9352
Epoch 68/70
Epoch 00068: val_loss did not improve
 - 5s - loss: 0.2541 - acc: 0.9017 - val_loss: 0.215

Epoch 52/70
Epoch 00052: val_loss improved from 0.24398 to 0.24359, saving model to best_m.h5
 - 5s - loss: 0.2140 - acc: 0.9172 - val_loss: 0.2436 - val_acc: 0.8928
Epoch 53/70
Epoch 00053: val_loss did not improve
 - 5s - loss: 0.2365 - acc: 0.9039 - val_loss: 0.2550 - val_acc: 0.8828
Epoch 54/70
Epoch 00054: val_loss did not improve
 - 5s - loss: 0.1948 - acc: 0.9225 - val_loss: 0.2600 - val_acc: 0.8953
Epoch 55/70
Epoch 00055: val_loss did not improve
 - 5s - loss: 0.2010 - acc: 0.9142 - val_loss: 0.2481 - val_acc: 0.8903
Epoch 56/70
Epoch 00056: val_loss did not improve
 - 5s - loss: 0.1982 - acc: 0.9200 - val_loss: 0.2498 - val_acc: 0.8978
Epoch 57/70
Epoch 00057: val_loss did not improve
 - 5s - loss: 0.2057 - acc: 0.9145 - val_loss: 0.2675 - val_acc: 0.8903
Epoch 58/70
Epoch 00058: val_loss did not improve
 - 5s - loss: 0.2416 - acc: 0.9045 - val_loss: 0.2684 - val_acc: 0.8878
Epoch 59/70
Epoch 00059: val_loss did not improve
 - 5s - loss: 0.2222 - acc: 0.9122 - val_loss: 0.253

Epoch 46/70
Epoch 00046: val_loss did not improve
 - 5s - loss: 0.2354 - acc: 0.9058 - val_loss: 0.2674 - val_acc: 0.8853
Epoch 47/70
Epoch 00047: val_loss did not improve
 - 5s - loss: 0.2080 - acc: 0.9092 - val_loss: 0.2851 - val_acc: 0.8803
Epoch 48/70
Epoch 00048: val_loss did not improve
 - 5s - loss: 0.2299 - acc: 0.9164 - val_loss: 0.2636 - val_acc: 0.8828
Epoch 49/70
Epoch 00049: val_loss did not improve
 - 5s - loss: 0.2126 - acc: 0.9217 - val_loss: 0.2677 - val_acc: 0.8903
Epoch 50/70
Epoch 00050: val_loss did not improve
 - 5s - loss: 0.1970 - acc: 0.9167 - val_loss: 0.2779 - val_acc: 0.8778
Epoch 51/70
Epoch 00051: val_loss did not improve
 - 5s - loss: 0.1986 - acc: 0.9208 - val_loss: 0.2779 - val_acc: 0.8803
Epoch 52/70
Epoch 00052: val_loss did not improve
 - 5s - loss: 0.2138 - acc: 0.9056 - val_loss: 0.2625 - val_acc: 0.8828
Epoch 53/70
Epoch 00053: val_loss did not improve
 - 5s - loss: 0.2001 - acc: 0.9175 - val_loss: 0.2710 - val_acc: 0.8728
Epoch 54/70
Epoch 00054:

Epoch 00039: val_loss did not improve
 - 5s - loss: 0.2999 - acc: 0.8889 - val_loss: 0.2978 - val_acc: 0.8778
Epoch 40/70
Epoch 00040: val_loss did not improve
 - 5s - loss: 0.2811 - acc: 0.8881 - val_loss: 0.2475 - val_acc: 0.9127
Epoch 41/70
Epoch 00041: val_loss did not improve
 - 5s - loss: 0.2778 - acc: 0.8875 - val_loss: 0.2611 - val_acc: 0.9202
Epoch 42/70
Epoch 00042: val_loss did not improve
 - 5s - loss: 0.2882 - acc: 0.8845 - val_loss: 0.2894 - val_acc: 0.8778
Epoch 43/70
Epoch 00043: val_loss did not improve
 - 5s - loss: 0.2898 - acc: 0.8883 - val_loss: 0.2444 - val_acc: 0.9052
Epoch 44/70
Epoch 00044: val_loss improved from 0.24216 to 0.23544, saving model to best_m.h5
 - 5s - loss: 0.2685 - acc: 0.8928 - val_loss: 0.2354 - val_acc: 0.9077
Epoch 45/70
Epoch 00045: val_loss did not improve
 - 5s - loss: 0.2901 - acc: 0.8925 - val_loss: 0.2816 - val_acc: 0.8953
Epoch 46/70
Epoch 00046: val_loss improved from 0.23544 to 0.21979, saving model to best_m.h5
 - 5s - loss: 0.2392